### Load CSV into Jupyter Notebook

In order to create the database without issue, we have to clean some of the data in the csv file.  First, we load and check the csv data

In [1]:

import pandas as pd

df = pd.read_csv("e_commerce_dataset.csv", encoding='ISO-8859-1')

df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


### Clean Data

In [2]:
print(df.columns.tolist())

['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']


We want to strip all of the dollar amounts of their dollar signs, as was the format in the excel file

Format dates to YYYY-MM-DD for MySQL

Ensure that the zip code is clean 5-digit number.  Some of the zip codes 

In [3]:
# Strip special chars from numeric columns
cols_to_clean = ['Sales', 'Profit', 'Discount']
for col in cols_to_clean:
    df[col] = df[col].astype(str).str.replace(r'[^0-9.\-]', '', regex=True).astype(float)
    df[col] = df[col].round(2)
# Format dates
df['Order Date'] = pd.to_datetime(df['Order Date']).dt.strftime('%Y-%m-%d')
df['Ship Date'] = pd.to_datetime(df['Ship Date']).dt.strftime('%Y-%m-%d')

# Clean postal code
df['Postal Code'] = df['Postal Code'].astype(str).str.zfill(5)

In [4]:
df.to_csv('cleaned_ecommerce.csv', index=False)

### Make a Connection and Load Cleaned Data into MySQL Workbench

In [5]:
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\orand\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
# Connect to MySQL Workbench
import mysql.connector

config = {
    'user': 'root',
    'password': 'root',
    'host': '127.0.0.1',
    'port': 3306,
    'database': 'ecommerce',
    'raise_on_warnings': True
}

mydb = mysql.connector.connect(**config)
my_cursor = mydb.cursor()

In [7]:
df = pd.read_csv('cleaned_ecommerce.csv')

In [8]:
insert_query = """
INSERT INTO ecommerce (
    Order_ID, Order_Date, Ship_Date, Ship_Mode, Customer_ID, Customer_Name,
    Segment, Country, City, State, Postal_Code, Region, Product_ID, Category,
    SubCategory, Product_Name, Sales, Quantity, Discount, Profit
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)
"""

In [9]:
data = df.iloc[:, 1:].values.tolist()  # Skip 'Row ID' column

One of the issues we found when trying to load data was that the size of the csv was too big to be loaded all at once, so we break it into smaller chunks

In [10]:
chunk_size = 1000
for start in range(0, len(data), chunk_size):
    end = start + chunk_size
    chunk = data[start:end]
    my_cursor.executemany(insert_query, chunk)
    mydb.commit()

print("Data successfully inserted into ecommerce table.")

Data successfully inserted into ecommerce table.


In [11]:
# check to make sure no duplicate values were added to the database in MySQL Workbench
print(len(df)) 

9994


In [13]:
# Check number of rows in the ecommerce database.  Should match with the length of the dataframe
my_cursor.execute("SELECT COUNT(*) FROM ecommerce;")
row_count = my_cursor.fetchone()[0]
print(row_count)

9994


In [20]:
# Sample Query 
# Clear any unread result sets
while my_cursor.nextset():
    pass

# Sample query
sql_query = "SELECT Product_ID FROM ecommerce LIMIT 5"
my_cursor.execute(sql_query)
rows = my_cursor.fetchall()

#Print as text
for row in rows:
    print(row)

('FUR-BO-10001798',)
('FUR-CH-10000454',)
('OFF-LA-10000240',)
('FUR-TA-10000577',)
('OFF-ST-10000760',)


### Checking for 1NF Normalization

In [21]:
# Check for atomicity
# Re-query a sample
my_cursor.execute("SELECT * FROM ecommerce")
rows = my_cursor.fetchall()
columns = [col[0] for col in my_cursor.description]

# Check for potential non-atomic values (like commas or semicolons)
for col_idx, col_name in enumerate(columns):
    for row in rows:
        val = str(row[col_idx])
        if ',' in val or ';' in val:
            print(f"Non-atomic value found in column '{col_name}': {val}")
            break

Non-atomic value found in column 'Product_Name': Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back


Only one value was found, and in this case, it's still atomic because the comma is just part of the description, not a separate item.

In [27]:
# Check for duplicate rows
from collections import Counter

duplicates = [item for item, count in Counter(rows).items() if count > 1]
if duplicates:
    print(f"Found {len(duplicates)} duplicate row(s).")
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [26]:
# Check for column value consistency
import collections

# Check column data types for first 100 rows
type_check = collections.defaultdict(set)
for row in rows[:100]:
    for idx, val in enumerate(row):
        type_check[columns[idx]].add(type(val).__name__)

# Only show columns with mixed types
mixed = {col: types for col, types in type_check.items() if len(types) > 1}

if mixed:
    print("Columns with mixed types detected:")
    for col, types in mixed.items():
        print(f"  - {col}: {types}")
else:
    print("All columns have consistent data types.")

All columns have consistent data types.
